# RAPIDS NLP:

### Credits:
https://medium.com/rapids-ai/show-me-the-word-count-3146e1173801


### Objective: Show case nlp capabilties of nvstrings+cudf

### Import libraries

In [1]:
import cudf
import nvcategory
import os
import numpy as np
import nvtext
import cuml
import nvstrings
try:
    import nltk
except ModuleNotFoundError:
    os.system('pip install nltk')
    import nltk
from numba import cuda

### Set Data File 

In [2]:
data_file = 'data/enwiki-latest.preprocess.txt.gz'
output_dir = 'data'

## Read Text Frame

#### Read helper functions

In [3]:
import gensim.utils
import tqdm
import sys

ONE_GB = 10 ** 8

def get_non_empty_lines(path):
    """
        returns non empty lines from a list of lines
    """
    chunked_output = []
    for line in get_txt_lines(path):
        chunked_output += [line]
        if sys.getsizeof(chunked_output) > ONE_GB:
            print('Chunk Size:', sys.getsizeof(chunked_output), 'bytes')
            print('yielding')
            yield chunked_output
            chunked_output = []
    yield chunked_output

def get_txt_lines(path):
    """
        Read text lines from gutenberg texts
        returns (text_ls,fname_ls) where 
        text_ls= input_text_lines and fname_ls = list of fnames
    """
    with gensim.utils.open(path, 'rb', encoding='utf-8') as infile:
        for i, line in tqdm.tqdm(enumerate(infile)):
            yield line

### Read text lines into a cudf dataframe

#### Full Pre-processing Pipe-Line

In [4]:
filters = [ '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', '-', '.', '/',  '\\', ':', ';', '<', '=', '>',
           '?', '@', '[', ']', '^', '_', '`', '{', '|', '}', '\~', '\t','\\n',"'",",",'~' , '—']

def preprocess_text(input_strs , filters=filters):
    """
        * filter punctuation
        * to_lower
        * remove stop words (from nltk corpus)
        * remove multiple spaces with one
        * remove leading spaces    
    """
    
    # filter punctuation and case conversion
    input_strs = input_strs.str.replace_multi(filters, ' ', regex=False)
    input_strs = input_strs.str.lower()
        
    # replace multiple spaces with single one and strip leading/trailing spaces
    input_strs = input_strs.str.replace(r"\s+", ' ', regex=True)
    input_strs = input_strs.str.strip(' ')
    
    return input_strs

def preprocess_text_df(df, text_cols=['text'], **kwargs):
    for col in text_cols:
        df[col] = preprocess_text(df[col], **kwargs)
    return  df

With our function defined, we can execute it to preprocess the entire dataset.

In [5]:
for i, txt_ls in enumerate(get_non_empty_lines(data_file)):
    df = cudf.DataFrame()

    df['text'] = nvstrings.to_device(txt_ls)
    print('chunk',i,': read into GPU')

    df = preprocess_text_df(df)
    print('chunk',i,': cleaned')

    df.to_csv(path=data_file+'.clean.'+str(i), columns=['text'], header=False, index=False)
    print('chunk',i,': written to file')


11463627it [00:51, 216029.61it/s]

Chunk Size: 103184048 bytes
yielding
chunk 0 : read into GPU


11463627it [01:10, 216029.61it/s]

chunk 0 : cleaned


11497620it [03:12, 60.60it/s]    

...

chunk 9 : cleaned


114691982it [1:05:28, 287.07it/s] 

chunk 9 : written to file


121169182it [1:05:58, 30613.34it/s] 


chunk 10 : read into GPU
chunk 10 : cleaned
chunk 10 : written to file
